In [1]:
import numpy as np

from scipy.optimize import minimize
from scipy.io import loadmat

from bokeh.plotting import show, figure
from bokeh.io import output_notebook
from bokeh.layouts import column, row, gridplot
                 
from math import nan, inf
                 
from time import time

In [2]:
output_notebook()

Loading BokehJS ...

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.nan_to_num(np.exp(-z)))


def get_nn_y(y, K):
    
    m = y.size
    
    tmp_y = np.copy(y)
    
    tmp_y = tmp_y.astype(np.int)
    
    nn_y = np.zeros((m, K))
    
    for i in range(m):
        nn_y[i, tmp_y[i]] = 1
    
    return nn_y


def get_theta_shapes(layers):
    
    L = len(layers)
    
    theta_shapes = []
    
    for i in range(L - 1):
        
        curr_size = layers[i]
        next_size = layers[i + 1]
        
        theta_shapes.append((next_size, curr_size + 1))
    
    return theta_shapes


def get_rand_thetas(layers):
    
    theta_shapes = get_theta_shapes(layers)
    
    L = len(layers)
    
    thetas = []
    
    for i in range(L - 1):
        
        eps = np.sqrt(6/(layers[i] + layers[i + 1]))
        
        dim1, dim2 = theta_shapes[i]
        
        thetas.append(np.random.rand(dim1, dim2) * 2 * eps - eps)
        
    return thetas


def flatten_thetas(thetas):
    
    T = len(thetas)
       
    flat_thetas = np.array([])
    
    for i in range(T):
        flat_thetas = np.append(flat_thetas, thetas[i].flatten())
    
    return flat_thetas


def deflatten_thetas(flat_thetas, layers):
    
    theta_shapes = get_theta_shapes(layers)
    
    T = len(theta_shapes)
    
    thetas = [None] * T
    
    prev_end = 0
    
    for i in range(T):
        
        end = theta_shapes[i][0] * theta_shapes[i][1] + prev_end 
        
        thetas[i] = np.reshape(flat_thetas[prev_end:end], theta_shapes[i])
        
        prev_end = end
        
    return thetas


def forward_prop(X, thetas):

    T = len(thetas)
    
    layer_values = []
    
    Prev_layer = np.copy(X)
    
    for i in range(T):
        
        Prev_layer = np.insert(Prev_layer, 0, 1, axis=1)
        
        layer_values.append(Prev_layer)
    
        Current_layer = sigmoid(Prev_layer.dot(thetas[i].transpose()))
        
        Prev_layer = Current_layer
                
    layer_values.append(Prev_layer)
    
    return layer_values
      

def nn_cost(flat_thetas, layers, X, y, lambd):
    
    thetas = deflatten_thetas(flat_thetas, layers)
    
    T = len(thetas)
    
    m, n = X.shape
        
    H = forward_prop(X, thetas)[-1]
    
    J = (-1/m) * np.sum(y * np.log(H) + (1 - y) * np.log(1 - H))
    
    for i in range(T):
        J += lambd/(2*m) * np.sum(np.square(thetas[i][:, 1:]))       
        
    return J


def back_prop(flat_thetas, layers, X, y, lambd):
    
        thetas = deflatten_thetas(flat_thetas, layers)
        
        theta_shapes = get_theta_shapes(layers)
        
        m, n = X.shape
        
        L = len(layers)
        
        T = len(thetas)
        
        thetas_grad = [None] * T
        
        for i in range(T):
            thetas_grad[i] = np.zeros(theta_shapes[i])
    
        layer_values = forward_prop(X, thetas) 
        
        deltas = [None] * L
        
        deltas[L - 1] = layer_values[L - 1] - y
        
        for i in range(L - 2, 0, -1): 
            
            d_sigmoid_i = layer_values[i] * (1 - layer_values[i])
            
            deltas[i] = deltas[i + 1].dot(thetas[i]) * d_sigmoid_i
            
            deltas[i] = deltas[i][:, 1:]              
                
        for i in range(T):
            
            Theta_tmp = np.copy(thetas[i])
            
            Theta_tmp[:, 0] = 0
            
            thetas_grad[i] = 1/m * ((deltas[i + 1].transpose().dot(layer_values[i])) + lambd * Theta_tmp)
                  
        return flatten_thetas(thetas_grad)


def back_prop_check(flat_thetas, flat_grad, layers, eps, X, y, lambd):
    
    theta_shapes = get_theta_shapes(layers)
    
    sum_err = 0
    
    t = flat_grad.size
    
    for i in range(t):
        
        theta_pls = np.copy(flat_thetas)
        theta_min = np.copy(flat_thetas)
        
        theta_pls[i] += eps
        theta_min[i] -= eps
        
        approx_grad = 1/(2*eps) * (nn_cost(theta_pls, layers, X, y, lambd) - nn_cost(theta_min, layers, X, y, lambd))
        
        sum_err += abs(approx_grad - flat_grad[i])
                
    return sum_err/t


def gradient_descent(flat_init_thetas, X, y, layers, lambd, alpha, iterations, get_plt_data = False):
    
    thetas = flat_init_thetas

    m, n = X.shape
    
    if get_plt_data:
        iter_x = np.arange(iterations)
        iter_y = np.zeros(iterations)
    
    for i in range(iterations):
        grad = back_prop(thetas, layers, X, y, lambd)
                            
        thetas -= alpha*grad
        
        if get_plt_data:
            iter_y[i] = nn_cost(thetas, layers, X, y, lambd)
      
    if get_plt_data:
        return flatten(thetas, layers), iter_x, iter_y
    
    else:
        return thetas, nn_cost(thetas, layers, X, y, lambd)


def train(X, y, layers, lambd, method, alpha = None, iterations = None):
    
    flat_init_thetas = flatten_thetas(get_rand_thetas(layers))
    
    if method == 'GD':
        trained_thetas, cost = gradient_descent(flat_init_thetas, X, y, layers, lambd, alpha, iterations)

    else:
        op_result = minimize(fun = nn_cost, 
                             x0 = flat_init_thetas, 
                             args = (layers, X, y, lambd),
                             method = method,
                             jac = back_prop)
        
        trained_thetas = op_result['x']
        cost = op_result['fun']
    
    return deflatten_thetas(trained_thetas, layers), cost

    
def predict(X, thetas):
    
    predictions = forward_prop(X, thetas)[-1]

    predictions = np.argmax(predictions, axis=1)
    
    return predictions
                

def error(X, thetas, answers):
    
    predictions = predict(X, thetas)
    
    wrong_answers = predictions[predictions != answers].size
    
    all_answers = answers.size
    
    return wrong_answers / all_answers


def accuracy(X, thetas, answers):
    return 100*(1 - error(X, thetas, answers)) 


def training_analysis(method, atr, atr_range,
                      X_train, y_train, nn_y_train,
                      X_val, y_val, 
                      no_of_inputs = None,
                      no_of_classes = None,
                      default_layers = None, 
                      default_lambd = None, 
                      default_alpha = None, 
                      default_iterations = None,
                      print_progress = False):
    
    
    comp_time = []
    final_cost = [] 
    val_err = []
    train_err = []
    all_thetas = []

    for i in atr_range:
        
        time_start = time()
        
        if atr == 'examples':
            thetas, cost = train(X_train[:i, :], 
                                 nn_y_train[:i, :], 
                                 default_layers, 
                                 default_lambd, 
                                 method,
                                 default_alpha, 
                                 default_iterations)
   
        if atr[2:] == 'hidden_layers_sizes':          
            no_of_layers = int(atr[0])
            
            tested_layers = [i for n in range(no_of_layers)]
            tested_layers = [no_of_inputs] + tested_layers + [no_of_classes]

            thetas, cost = train(X_train, 
                                 nn_y_train, 
                                 tested_layers,
                                 default_lambd,
                                 method,
                                 default_alpha, 
                                 default_iterations)
       
        if atr == 'lambd':
            thetas, cost = train(X_train, 
                                 nn_y_train,
                                 default_layers,
                                 i, 
                                 method,
                                 default_alpha, 
                                 default_iterations) 
            
        if atr == 'alpha':
            thetas, cost = train(X_train, 
                                 nn_y_train,
                                 default_layers,
                                 default_lambd,
                                 method,
                                 i, 
                                 default_iterations)

        if atr == 'max_iterations':
            thetas, cost = train(X_train, 
                                 nn_y_train,
                                 default_layers,
                                 default_lambd,
                                 method,
                                 default_alpha, 
                                 i)

        time_end = time()
        time_elapsed = time_end - time_start

        
        tmp_X_train = X_train[:i, :] if atr == "examples" else X_train
        tmp_y_train = y_train[:i, :] if atr == "examples" else y_train
        tmp_X_val = X_train[i:i + int(3/7*i), :] if atr == "examples" else X_val
        tmp_y_val = y_train[i:i + int(3/7*i), :] if atr == "examples" else y_val
                       
        train_err_measured = error(tmp_X_train, thetas, tmp_y_train)
        val_err_measured = error(tmp_X_val, thetas, tmp_y_val)
        
        
        final_cost.append(cost)
        comp_time.append(time_elapsed)
        val_err.append(val_err_measured)
        train_err.append(train_err_measured)
        all_thetas.append(thetas)
        
        if print_progress:
            print "cost: {}, time: {}, val_err: {}".format(cost, time_elapsed, val_err_measured)
    
    return [('method', method),
            (atr,  atr_range), 
            ('computation_time[s]', comp_time), 
            ('final_cost', final_cost),  
            ('val_error', val_err),
            ('train_error', train_err)], all_thetas


def get_plots(plot_data_list, legend_ad = '', color = "blue"):
    
    all_legend = plot_data_list[0][1] + legend_ad    
    plots = []

    all_x_axis = plot_data_list[1][1]
    all_x_axis_label = plot_data_list[1][0]
    
    for plot_data in plot_data_list[2:-1]:
        
        plots.append(figure(x_axis_label = all_x_axis_label , 
                             y_axis_label = plot_data[0]))
        
        plots[-1].line(x = all_x_axis, 
                       y = plot_data[1],
                       legend = all_legend,
                       color = color)
        
    err_plot = figure(x_axis_label = all_x_axis_label,
                      y_axis_label = plot_data_list[-1][0])
    
    err_plot.line(x = all_x_axis,
                  y = plot_data_list[-2][1],
                  legend = plot_data_list[-2][0],
                  color = color)
    
    err_plot.line(x = all_x_axis,
                  y = plot_data_list[-1][1],
                  legend = plot_data_list[-1][0],
                  color = 'green')
    
    return plots, err_plot


def add_to_plots(plots, plot_data_list, legend_ad = '', color = "firebrick"):
    
    all_legend = plot_data_list[0][1] + legend_ad
    
    all_x_axis = plot_data_list[1][1]
    
    plt_iter = iter(plots)
    
    for plot_data in plot_data_list[2:-1]:

        next(plt_iter).line(x = all_x_axis, 
                       y = plot_data[1],
                       legend = all_legend,
                       color = color)
    

In [96]:
#raw_data must be in a form - atributes in columns 0:-1, classification in column -1
def extract_raw_data(raw_data):
    raw_data = np.copy(raw_data)
    
    all_examples = raw_data.shape[0]
    
    train_examples = int(0.6 * all_examples)
    val_examples = int(0.2 * all_examples)
    test_examples = all_examples - (train_examples + val_examples)
    
    X_train = raw_data[:train_examples, :-1]
    y_train = raw_data[:train_examples, -1]
    
    X_val = raw_data[train_examples: train_examples + val_examples , :-1]
    y_val = raw_data[train_examples: train_examples + val_examples , -1]
    
    X_test = raw_data[train_examples + val_examples: , :-1]
    y_test = raw_data[train_examples + val_examples: , -1]
    
    return X_train, y_train, X_val, y_val, X_test, y_test